In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image

import numpy as np
import tensorflow as tf
from tensorflow import keras

import warnings;
warnings.filterwarnings('ignore')

# Predict batches of images

In [ ]:
tf.compat.v1.enable_v2_behavior()

In [ ]:
label = ['3_24+10', '3_24+30', '3_24+5', '3_24+60', '3_24+70', '3_24+90', '3_24+110', '3_24+20', '3_24+40', '3_24+50', '3_24+80', '1_12_1', '1_12_2', '1_13', '1_14', '1_19', '1_24', '1_26', '1_27', '3_21', '3_31', '3_33', '4_4_1', '4_4_2', '4_5_2', '4_5_4', '4_5_5', '4_8_5', '4_8_6', '5_17', '6_2+50', '6_2+70', '6_2+30', '6_2+40', '6_2+60', '6_2+80', '6_7', '7_1', '7_11', '7_13', '7_14', '7_2', '7_4', '7_7', '7_9', 'smoke', 'unknown', '1_11_1', '1_11_2', '1_15', '1_16', '1_18', '1_20_1', '1_22', '1_25', '1_28', '1_29', '1_30', '1_8', '2_3_1', '2_3_L', '2_3_R', '2_6', '2_7', '3_15', '3_17', '3_20', '3_25+70', '3_25+20', '3_25+30', '3_25+40', '3_25+50', '3_25+5', '3_25+60', '3_6', '4_1_6', '4_2_1', '4_2_2', '5_15_5', '6_3_1', '7_3', '7_6', '1_17', '3_16', '5_15_3', '5_20', '7_12', '1_31', '3_10', '3_19', '3_2', '3_5', '3_7', '3_9', '4_1_2_1', '4_1_3_1', '4_5_1', '4_5_6', '4_8_1', '4_8_2', '4_8_3', '5_1', '5_11_1', '5_12_1', '5_13_1', '5_13_2', '5_14_1', '5_14_2', '5_14_3', '5_2', '5_23_2', '5_24_2', '5_3', '5_4', '5_8', '7_5', '3_32', '7_18', '1_2', '1_33', '1_7', '2_4', '3_18_1', '3_18_2', '3_8', '4_1_2', '4_1_3', '5_14', '6_15_2', '6_15_3', '6_6', '6_8_1', '1_1', '1_20_2', '1_20_3', '1_21', '1_23', '1_5', '2_1', '2_2', '2_5', '3_1', '3_26', '3_27', '3_28', '3_29', '3_30', '4_1_1', '4_1_4', '4_1_5', '4_2_3', '4_3', '4_8_4', '5_16', '5_18', '5_19', '5_21', '5_22', '5_5', '5_6', '5_7_1', '5_7_2', '5_9', '6_15_1', '6_16', '6_4', '6_8_2', '6_8_3', '5_29', '5_31+10', '5_31+20', '5_31+30', '5_31+40', '5_31+5', '5_31+50', '5_32', '5_33', '1_6', '5_15_2+2', '5_15_2+1', '5_15_2+3', '5_15_2+5']

In [ ]:
autoencoder = keras.models.load_model("../input/aaaaaaaaaa/autoencoder.h5") # load pre_trained auto encoder model

In [ ]:
model_1= keras.models.load_model("../input/aaaaaaaaaa/VGG19_2.h5")
model_2= keras.models.load_model("../input/aaaaaaaaaa/InceptionResNetV2_2.h5")
model_3 = keras.models.load_model('../input/aaaaaaaaaa/denset201_2.h5')

In [ ]:
root_dir = '../input/aiijc-final-dcm/AIJ_2gis_data/'
def load_and_change_img(img):
    img = image.img_to_array(img)
    img = img/255.
    result= autoencoder.predict(img[None])
    new_arr = ((result - result.min()) * (1/(result.max() - result.min()) * 255)).astype('uint8')
    img_new = np.zeros(shape=(80,80,3), dtype= np.int16)
    img_new[..., 0] = new_arr[...,2]
    img_new[...,1]=new_arr[...,1]
    img_new[..., 2] = new_arr[...,0]
    return img_new/255.

In [ ]:
df = pd.read_csv("../input/aiijc-final-dcm/AIJ_2gis_data/sample_submission.csv")

In [ ]:
df_a=df[0:100000]

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=load_and_change_img)
test_set =train_datagen.flow_from_dataframe(directory = root_dir,
                                            dataframe=df_a,
                                            x_col = 'filename',
                                            y_col='label',
                                            classes=None,
                                            class_model=None,
                                            shuffle=False,
                                            batch_size=256,
                                            target_size=(80,80))

In [ ]:
outputs=[]
y_pred_1=model_1.predict(test_set, batch_size=256,verbose=1)
y_pred_2=model_2.predict(test_set, batch_size=256,verbose=1)
y_pred_3=model_3.predict(test_set, batch_size=256, verbose=1)
y_pred=y_pred_1*0.2 + y_pred_2*0.4 + y_pred_3*0.4
del y_pred_1
del y_pred_2
del y_pred_3
for i in range(len(np.argmax(y_pred, axis=1))):
    outputs.append(label[np.argmax(y_pred[i], axis=0)])

In [ ]:
df_new=pd.DataFrame({'filename': df_a['filename'], 'label': outputs})
df_new.to_csv('predict.csv')

# Predict single image

In [ ]:
model1= keras.models.load_model("../input/aaaaaaaaaa/VGG19_2.h5")
model2= keras.models.load_model("../input/aaaaaaaaaa/InceptionResNetV2_2.h5")
model3 = keras.models.load_model('../input/aaaaaaaaaa/denset201_2.h5')

In [ ]:
def auto_encoder(img_path):
  img = image.load_img(img_path, target_size=(80,80,3))
  img = image.img_to_array(img)
  img = img/255.
  result= autoencoder.predict(img[None])
  new_arr = ((result - result.min()) * (1/(result.max() - result.min()) * 255)).astype('uint8')
  img_new = np.zeros(shape=(80,80,3), dtype=np.int16)
  img_new[..., 0] = new_arr[...,2]
  img_new[...,1]=new_arr[...,1]
  img_new[..., 2] = new_arr[...,0]
  return img_new/255.

In [ ]:
labels=[]
img_path=""
def predict(img_path):
  img = auto_encoder(img_path)
  y_pred1=model1.predict(np.expand_dims(img, axis=0)*1/255.0)
  y_pred2=model2.predict(np.expand_dims(img, axis=0)*1/255.0)
  y_pred3=model3.predict(np.expand_dims(img, axis=0)*1/255.0)
  y_pred=y_pred1*0.2 + y_pred2*0.4 + y_pred3*0.4
  print(label[np.argmax(y_pred)])